In [2]:
import os
import astroNN
from astroNN import datasets
from astroNN.apogee import allstar, visit_spectra, combined_spectra, gap_delete, apogee_continuum, bitmask_boolean
from astroNN.models import load_folder
from astroNN.apogee.chips import wavelength_solution, chips_split

from astropy.io import fits
from astropy.wcs import WCS
from tensorflow.keras import utils
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle
import traceback

%matplotlib inline
import matplotlib.ticker as ticker

2023-06-11 21:18:31.818310: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 21:18:34.263434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
geir_path = "/geir_data/scr/petershi" #should save work here 
sdssv_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star" #path for SDSSV spectra
sdssiv_path = "/yngve_data/sdss/apogee/dr17/apogee/spectro/redux/dr17/stars/apo25m"

xmatch_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/aux_xmatch/ipl-1_dr17_lite_230517_byYS.fits"

model_path = "/home/petershi/MyFiles/astroNN_spectra_paper_figures/astroNN_0617_run001"
allstar_path = "/yngve_data/sdss/apogee/dr17/apogee/spectro/aspcap/dr17/synspec/allStar-dr17-synspec.fits"

# Exploring Data Structure


In [8]:
hdul = fits.open(allstar_path, memmap = False)
hdul.info()

Filename: /yngve_data/sdss/apogee/dr17/apogee/spectro/aspcap/dr17/synspec/allStar-dr17-synspec.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   ()      
  1                1 BinTableHDU    495   733901R x 234C   [64A, 30A, 58A, 71A, 69A, 6A, J, 20A, 30A, D, D, D, D, E, E, E, E, E, E, 16A, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, J, J, E, E, 16A, E, 32A, E, E, J, J, J, J, J, J, 132A, 32A, 32A, J, E, E, K, 132A, K, 132A, E, E, E, E, E, E, E, E, E, E, E, J, J, E, E, E, E, E, E, K, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, 8A, 189E, 21E, 9E, 81E, E, 9E, 81E, 9K, K, 256A, E, E, E, 27E, 27D, 27E, 27E, 27E, 27E, 27E, 27E, 27K, J, K, 10A, 27E, 27E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E, E, E, J, E,

In [9]:
print(repr(hdul[1].header))

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 5402 / length of dimension 1                          
NAXIS2  =               733901 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  234 / number of table fields                         
TTYPE1  = 'FILE    '                                                            
TFORM1  = '64A     '                                                            
TTYPE2  = 'APOGEE_ID'                                                           
TFORM2  = '30A     '                                                            
TTYPE3  = 'TARGET_ID'       

In [18]:
print(repr(hdul[1].data[hdul[1].data["APOGEE_ID"] == "VESTA"]["X_H_err"][0]))

array([0.02953799, 0.02859265, 0.02302215, 0.04933612, 0.11133619,
       0.01149605, 0.01362693, 0.01180114, 0.08544085, 0.04145411,
       0.03267189, 0.01081002, 0.05619248, 0.24939212, 0.09601425,
       0.04869533, 0.0114673 , 0.00540092, 0.20884213, 0.0102655 ,
       0.05940297, 1.        , 1.        , 0.12798055, 0.07002651,
       1.        , 0.15617779], dtype=float32)


In [15]:
print(repr(hdul[1].data["APOGEE_ID"]))

chararray(['VESTA', '2M00000002+7417074', '2M00000019-1924498', ...,
           '2M23595921+5609479', '2M23595949-7342592',
           '2M23595980+1528407'], dtype='<U30')


In [18]:
print(repr(hdul[1].data["APOGEE_ID"] == '2M00000002+7417074'))

array([False,  True, False, ..., False, False, False])


In [17]:
print(repr(hdul[1].data["teff"]))

array([5694.8984, 3723.9111, 5501.773 , ..., 4585.4097, 4505.6235,
       6281.627 ], dtype=float32)


In [17]:
print(repr(hdul[1].data["logg_err"]))

array([0.01997363, 0.02152308, 0.02455476, ..., 0.02868619, 0.07665637,
       0.03876989], dtype=float32)


In [16]:
print(repr(hdul[1].data["Fe_H_err"]))

array([0.00540092, 0.0092513 , 0.00725273, ..., 0.01162025, 0.02089649,
       0.00922137], dtype=float32)


In [14]:
print(repr(hdul[1].data["X_H"]))

array([ 0.0083094 ,  0.0147425 ,  0.1277274 ,  0.1184003 ,  0.1501306 ,
        0.0386099 ,  0.078684  ,  0.0706517 ,         nan, -0.0859941 ,
       -0.0910071 , -0.0133666 , -0.22801359,  0.1605534 ,  0.09771055,
       -0.0223117 ,  0.04433251,  0.0034626 ,  0.188713  ,  0.054741  ,
               nan,         nan,         nan, -0.0014399 ,         nan,
               nan,         nan], dtype=float32)


In [15]:
print(repr(hdul[3].data["ELEM_SYMBOL"][0]))

chararray(['C', 'CI', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K',
           'Ca', 'Ti', 'TiII', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu',
           'Ge', 'Rb', 'Ce', 'Nd', 'Yb', 'C13'], dtype='<U20')


In [7]:
hdul.close()

# Compilation of Data


In [19]:
xmatch = fits.open(xmatch_path)
apo = xmatch[1].data["APOGEE_ID"]

In [20]:
hdul = fits.open(allstar_path)
rd = {}
symbols = hdul[3].data["ELEM_SYMBOL"][0]

for a in tqdm(apo):
    figs = {}
    fig_errs = {}

    adata = hdul[1].data[hdul[1].data["APOGEE_ID"] == a]

    teff = adata["teff"][0]
    logg = adata["logg"][0]
    feh = adata["Fe_H"][0]

    figs["teff"] = teff
    figs["logg"] = logg
    figs["Fe"] = feh

    teff_err = adata["teff_err"][0]
    logg_err = adata["logg_err"][0]
    feh_err = adata["Fe_H_err"][0]

    fig_errs["teff"] = teff_err
    fig_errs["logg"] = logg_err
    fig_errs["Fe"] = feh_err
    
    xh = adata["X_H"][0]
    xh_err = adata["X_H_err"][0]

    for sym, x, err in zip(symbols, xh, xh_err):
        figs[sym] = x
        fig_errs[sym] = err

    rd[a] = {"Points" : figs,
             "Errors" : fig_errs}


100%|██████████| 27571/27571 [57:37<00:00,  7.97it/s]  


In [21]:
rd

{'2M02222833-0609582': {'Points': {'teff': 6174.623,
   'logg': 3.8659608,
   'Fe': 0.12513,
   'C': 0.11715299,
   'CI': 0.119972095,
   'N': 0.110195994,
   'O': 0.13452357,
   'Na': 0.0812335,
   'Mg': 0.04775849,
   'Al': 0.286916,
   'Si': 0.24246599,
   'P': nan,
   'S': 0.121136494,
   'K': 0.1320709,
   'Ca': 0.06314069,
   'Ti': -0.26493102,
   'TiII': -0.53117806,
   'V': 0.08817055,
   'Cr': -0.15326391,
   'Mn': 0.17358251,
   'Co': 0.677514,
   'Ni': 0.092515,
   'Cu': nan,
   'Ge': nan,
   'Rb': nan,
   'Ce': 0.1287231,
   'Nd': nan,
   'Yb': nan,
   'C13': nan},
  'Errors': {'teff': 34.447742,
   'logg': 0.019264188,
   'Fe': 0.00440508,
   'C': 0.040723193,
   'CI': 0.029286405,
   'N': 0.1146894,
   'O': 0.093840756,
   'Na': 0.12923124,
   'Mg': 0.010720406,
   'Al': 0.012161969,
   'Si': 0.0115177585,
   'P': 0.109039776,
   'S': 0.036926992,
   'K': 0.03782505,
   'Ca': 0.010063248,
   'Ti': 0.078032084,
   'TiII': 0.38422897,
   'V': 0.13296723,
   'Cr': 0.05510973

In [22]:
with open(os.path.join(geir_path, "serialized", "SDSS-IV elementals.pkl"), "wb") as f:
    pickle.dump(rd, f)

In [23]:
hdul.close()